In [2]:
# Step 0: Read the correct latest merged hourly dataset
import pandas as pd

# Load dataset
merged_data = pd.read_parquet('../datasets/final_merged_dataset_hourly.parquet')

# Confirm the dataset is loaded
print(merged_data.head())
print(merged_data.columns)
print(f"Total data points (hours): {len(merged_data)}")


             datetime      Open      High       Low     Close    Volume  \
0 2021-01-01 00:00:00  0.250730  0.251106  0.250628  0.250493  0.076344   
1 2021-01-01 01:00:00  0.251433  0.251695  0.251353  0.251520  0.015258   
2 2021-01-01 02:00:00  0.256154  0.256011  0.255263  0.255877  0.034476   
3 2021-01-01 03:00:00  0.252147  0.252928  0.252351  0.253123  0.010795   
4 2021-01-01 04:00:00  0.254932  0.255284  0.254180  0.254066  0.023016   

   sentiment_score  bert_sentiment  prev_close  prev_vader_sentiment  \
0              0.0             0.0    0.250493                   0.0   
1              0.0             0.0    0.250493                   0.0   
2              0.0             0.0    0.251520                   0.0   
3              0.0             0.0    0.255877                   0.0   
4              0.0             0.0    0.253123                   0.0   

   prev_bert_sentiment  volatility_7d  volatility_14d  volatility_30d  \
0                  0.0       0.050683      

In [3]:
# Step 0: Read the correct latest merged hourly dataset
import pandas as pd

# Load dataset
merged_data = pd.read_parquet('../datasets/final_merged_dataset_hourly.parquet')

# Confirm the dataset is loaded
print(merged_data.head())
print(merged_data.columns)
print(f"Total data points (hours): {len(merged_data)}")


             datetime      Open      High       Low     Close    Volume  \
0 2021-01-01 00:00:00  0.250730  0.251106  0.250628  0.250493  0.076344   
1 2021-01-01 01:00:00  0.251433  0.251695  0.251353  0.251520  0.015258   
2 2021-01-01 02:00:00  0.256154  0.256011  0.255263  0.255877  0.034476   
3 2021-01-01 03:00:00  0.252147  0.252928  0.252351  0.253123  0.010795   
4 2021-01-01 04:00:00  0.254932  0.255284  0.254180  0.254066  0.023016   

   sentiment_score  bert_sentiment  prev_close  prev_vader_sentiment  \
0              0.0             0.0    0.250493                   0.0   
1              0.0             0.0    0.250493                   0.0   
2              0.0             0.0    0.251520                   0.0   
3              0.0             0.0    0.255877                   0.0   
4              0.0             0.0    0.253123                   0.0   

   prev_bert_sentiment  volatility_7d  volatility_14d  volatility_30d  \
0                  0.0       0.050683      

In [4]:
# Step 1 (updated): Set datetime as index
merged_data['datetime'] = pd.to_datetime(merged_data['datetime'])  # Ensure proper datetime format
merged_data = merged_data.set_index('datetime')

# Now extract Close prices
btc_close = merged_data['Close']

# Sanity check
print(btc_close.head())
print(f"Total hours of data: {len(btc_close)}")


datetime
2021-01-01 00:00:00    0.250493
2021-01-01 01:00:00    0.251520
2021-01-01 02:00:00    0.255877
2021-01-01 03:00:00    0.253123
2021-01-01 04:00:00    0.254066
Name: Close, dtype: float64
Total hours of data: 18265


In [ ]:
from statsmodels.tsa.stattools import adfuller

# Step 2: Perform ADF Test on btc_close
result = adfuller(btc_close)

print("ADF Statistic:", result[0])
print("p-value:", result[1])
for key, value in result[4].items():
    print(f"Critical Value ({key}): {value}")




ADF Statistic: -1.3083240084755148
p-value: 0.6253555173021281
Critical Value (1%): -3.4307086827260513
Critical Value (5%): -2.861698524313042
Critical Value (10%): -2.5668543783071662


### Non stationary dataaa

In [6]:
# Step 3: Apply first-order differencing
btc_close_diff = btc_close.diff().dropna()

# Re-run ADF test after differencing
result_diff = adfuller(btc_close_diff)

print("After Differencing - ADF Statistic:", result_diff[0])
print("After Differencing - p-value:", result_diff[1])
for key, value in result_diff[4].items():
    print(f"Critical Value ({key}): {value}")


After Differencing - ADF Statistic: -25.82320214571103
After Differencing - p-value: 0.0
Critical Value (1%): -3.4307086827260513
Critical Value (5%): -2.861698524313042
Critical Value (10%): -2.5668543783071662


### Now the data is stationary

In [7]:
import pmdarima as pm

# Step 4: Auto ARIMA to find best p and q
auto_model = pm.auto_arima(
    btc_close,
    start_p=0, start_q=0,
    max_p=5, max_q=5,
    d=1,
    seasonal=False,
    trace=True,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True
)

print(auto_model.summary())


/opt/anaconda3/envs/bit_pred_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/bit_pred_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-134303.582, Time=1.00 sec


/opt/anaconda3/envs/bit_pred_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-134302.237, Time=0.86 sec


/opt/anaconda3/envs/bit_pred_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-134302.256, Time=0.89 sec


/opt/anaconda3/envs/bit_pred_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-134305.565, Time=0.33 sec


/opt/anaconda3/envs/bit_pred_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-134302.048, Time=1.66 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 4.754 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                18265
Model:               SARIMAX(0, 1, 0)   Log Likelihood               67153.782
Date:                Thu, 17 Apr 2025   AIC                        -134305.565
Time:                        22:04:32   BIC                        -134297.752
Sample:                    01-01-2021   HQIC                       -134302.997
                         - 02-01-2023                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2      3.748e-05   1.33e-07    280.952      0.000    3.72e-

### ARIMA(1,1,0)

In [8]:
from statsmodels.tsa.arima.model import ARIMA

# Step 5: Train ARIMA(0,1,0)
model = ARIMA(btc_close, order=(0,1,0))
model_fit = model.fit()

# Print model summary
print(model_fit.summary())


/opt/anaconda3/envs/bit_pred_env/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/envs/bit_pred_env/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/envs/bit_pred_env/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                18265
Model:                 ARIMA(0, 1, 0)   Log Likelihood               67153.782
Date:                Thu, 17 Apr 2025   AIC                        -134305.565
Time:                        22:05:39   BIC                        -134297.752
Sample:                    01-01-2021   HQIC                       -134302.997
                         - 02-01-2023                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2      3.748e-05   1.33e-07    280.952      0.000    3.72e-05    3.77e-05
Ljung-Box (L1) (Q):                   0.66   Jarque-Bera (JB):            177856.79
Prob(Q):                              0.42   Pr

In [9]:
# Forecast next 30 steps (hours)
n_forecast_steps = 30
forecast = model_fit.forecast(steps=n_forecast_steps)

# Display forecasted values
print(forecast)


2023-02-01 01:00:00    0.142479
2023-02-01 02:00:00    0.142479
2023-02-01 03:00:00    0.142479
2023-02-01 04:00:00    0.142479
2023-02-01 05:00:00    0.142479
2023-02-01 06:00:00    0.142479
2023-02-01 07:00:00    0.142479
2023-02-01 08:00:00    0.142479
2023-02-01 09:00:00    0.142479
2023-02-01 10:00:00    0.142479
2023-02-01 11:00:00    0.142479
2023-02-01 12:00:00    0.142479
2023-02-01 13:00:00    0.142479
2023-02-01 14:00:00    0.142479
2023-02-01 15:00:00    0.142479
2023-02-01 16:00:00    0.142479
2023-02-01 17:00:00    0.142479
2023-02-01 18:00:00    0.142479
2023-02-01 19:00:00    0.142479
2023-02-01 20:00:00    0.142479
2023-02-01 21:00:00    0.142479
2023-02-01 22:00:00    0.142479
2023-02-01 23:00:00    0.142479
2023-02-02 00:00:00    0.142479
2023-02-02 01:00:00    0.142479
2023-02-02 02:00:00    0.142479
2023-02-02 03:00:00    0.142479
2023-02-02 04:00:00    0.142479
2023-02-02 05:00:00    0.142479
2023-02-02 06:00:00    0.142479
Freq: h, Name: predicted_mean, dtype: fl

In [11]:
merged_data

,Open,High,Low,Close,Volume,sentiment_score,bert_sentiment,prev_close,prev_vader_sentiment,prev_bert_sentiment,volatility_7d,volatility_14d,volatility_30d,closing_7d_avg,closing_30d_avg
datetime,,,,,,,,,,,,,,,
2021-01-01 00:00:00,0.250730,0.251106,0.250628,0.250493,0.076344,0.0,0.0,0.250493,0.0,0.0,0.050683,0.068865,0.057312,0.320975,0.357583
2021-01-01 01:00:00,0.251433,0.251695,0.251353,0.251520,0.015258,0.0,0.0,0.250493,0.0,0.0,0.050683,0.068865,0.057312,0.320975,0.357583
2021-01-01 02:00:00,0.256154,0.256011,0.255263,0.255877,0.034476,0.0,0.0,0.251520,0.0,0.0,0.050683,0.068865,0.057312,0.320975,0.357583
2021-01-01 03:00:00,0.252147,0.252928,0.252351,0.253123,0.010795,0.0,0.0,0.255877,0.0,0.0,0.050683,0.068865,0.057312,0.320975,0.357583
2021-01-01 04:00:00,0.254932,0.255284,0.254180,0.254066,0.023016,0.0,0.0,0.253123,0.0,0.0,0.050683,0.068865,0.057312,0.320975,0.357583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-31 20:00:00,0.141639,0.141700,0.141621,0.141516,0.001596,0.0,0.0,0.141875,0.0,0.0,0.005695,0.016235,0.049219,0.140301,0.085659
2023-01-31 21:00:00,0.141620,0.141530,0.141734,0.141648,0.001272,0.0,0.0,0.141516,0.0,0.0,0.005692,0.016183,0.049198,0.140323,0.085832
2023-01-31 22:00:00,0.141281,0.141285,0.141357,0.141403,0.006099,0.0,0.0,0.141648,0.0,0.0,0.005685,0.016131,0.049175,0.140352,0.086004


In [12]:
# Step 1: Make sure the index is sorted (important)
merged_data = merged_data.sort_index()

# Step 2: Define number of steps
n_forecast_steps = 30

# Step 3: Find the last timestamp
last_training_time = merged_data.index[-n_forecast_steps-1]

# Step 4: Get actual Close values for next 30 hours
actuals = merged_data[merged_data.index > last_training_time]['Close'].head(n_forecast_steps).values

# Step 5: Display actual values
print(actuals)


[0.14013819 0.14430987 0.14242223 0.14247886 0.14076111 0.1416483
 0.13464517 0.13353146 0.13417326 0.13404112 0.13607977 0.13602314
 0.13607977 0.1364573  0.13611752 0.13651393 0.13485281 0.13577775
 0.13872246 0.1374955  0.13713685 0.13579662 0.13674044 0.13753325
 0.14187482 0.14151617 0.1416483  0.14140291 0.14181819 0.14247886]


In [13]:
# Step 4: Forecast next 30 time steps using ARIMA model
forecast = model_fit.forecast(steps=n_forecast_steps)

# Display forecasted values
print(forecast)


2023-02-01 01:00:00    0.142479
2023-02-01 02:00:00    0.142479
2023-02-01 03:00:00    0.142479
2023-02-01 04:00:00    0.142479
2023-02-01 05:00:00    0.142479
2023-02-01 06:00:00    0.142479
2023-02-01 07:00:00    0.142479
2023-02-01 08:00:00    0.142479
2023-02-01 09:00:00    0.142479
2023-02-01 10:00:00    0.142479
2023-02-01 11:00:00    0.142479
2023-02-01 12:00:00    0.142479
2023-02-01 13:00:00    0.142479
2023-02-01 14:00:00    0.142479
2023-02-01 15:00:00    0.142479
2023-02-01 16:00:00    0.142479
2023-02-01 17:00:00    0.142479
2023-02-01 18:00:00    0.142479
2023-02-01 19:00:00    0.142479
2023-02-01 20:00:00    0.142479
2023-02-01 21:00:00    0.142479
2023-02-01 22:00:00    0.142479
2023-02-01 23:00:00    0.142479
2023-02-02 00:00:00    0.142479
2023-02-02 01:00:00    0.142479
2023-02-02 02:00:00    0.142479
2023-02-02 03:00:00    0.142479
2023-02-02 04:00:00    0.142479
2023-02-02 05:00:00    0.142479
2023-02-02 06:00:00    0.142479
Freq: h, Name: predicted_mean, dtype: fl

In [14]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

# Step 5: Calculate RMSE and MAPE
rmse = np.sqrt(mean_squared_error(actuals, forecast))
mape = mean_absolute_percentage_error(actuals, forecast) * 100  # in percentage

# Display Results
print(f"RMSE: {rmse:.6f}")
print(f"MAPE: {mape:.2f}%")


RMSE: 0.005184
MAPE: 3.13%


In [16]:
import sys
sys.path.append("../src")

from metric_logging import log_experiment


In [18]:
params = {
    "order": "(0, 1, 0)",
    "seasonal_order": "(0, 0, 0, 0)",  # No seasonal order for normal ARIMA
}

metrics = {
    "RMSE": rmse,
    "MAPE": mape,
}

log_experiment("ARIMA_model_for_hourly_data", model_object=model_fit, params=params, metrics=metrics)


2025/04/17 23:46:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ARIMA_model_for_hourly_data_Model'.
2025/04/17 23:46:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ARIMA_model_for_hourly_data_Model, version 1
Created version '1' of model 'ARIMA_model_for_hourly_data_Model'.


✅ ARIMA_model_for_hourly_data registered and logged successfully to DagsHub MLflow.
🏃 View run ARIMA_model_for_hourly_data-Baseline at: https://dagshub.com/vamsisaigarapati/bitcoin_price_pred_CSE574.mlflow/#/experiments/0/runs/9d373da8ab0c4013b47ee1efe7cb43eb
🧪 View experiment at: https://dagshub.com/vamsisaigarapati/bitcoin_price_pred_CSE574.mlflow/#/experiments/0


## SARIMA model

In [19]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Step 1: Define SARIMA model order
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 24)  # 24 hours seasonality (daily seasonality)

# Step 2: Fit SARIMA model
sarima_model = SARIMAX(
    merged_data['Close'],
    order=order,
    seasonal_order=seasonal_order,
    enforce_stationarity=False,
    enforce_invertibility=False
)
sarima_result = sarima_model.fit(disp=False)

# Step 3: Summary of model
print(sarima_result.summary())


/opt/anaconda3/envs/bit_pred_env/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/envs/bit_pred_env/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)


                                     SARIMAX Results                                      
Dep. Variable:                              Close   No. Observations:                18265
Model:             SARIMAX(1, 1, 1)x(1, 1, 1, 24)   Log Likelihood               66919.304
Date:                            Thu, 17 Apr 2025   AIC                        -133828.609
Time:                                    23:48:00   BIC                        -133789.559
Sample:                                01-01-2021   HQIC                       -133815.775
                                     - 02-01-2023                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0507      0.431      0.118      0.906      -0.794       0.895
ma.L1         -0.0614      0.431   

In [22]:
# Define forecast horizon
n_forecast_steps = 30

# Forecast using SARIMA
sarima_forecast = sarima_result.get_forecast(steps=n_forecast_steps)
sarima_forecast_values = sarima_forecast.predicted_mean

# Display the forecasted values
print(sarima_forecast_values)


2023-02-01 01:00:00    0.142523
2023-02-01 02:00:00    0.143036
2023-02-01 03:00:00    0.142825
2023-02-01 04:00:00    0.142813
2023-02-01 05:00:00    0.142790
2023-02-01 06:00:00    0.142956
2023-02-01 07:00:00    0.142683
2023-02-01 08:00:00    0.142395
2023-02-01 09:00:00    0.142450
2023-02-01 10:00:00    0.142532
2023-02-01 11:00:00    0.142470
2023-02-01 12:00:00    0.142556
2023-02-01 13:00:00    0.142520
2023-02-01 14:00:00    0.142573
2023-02-01 15:00:00    0.142502
2023-02-01 16:00:00    0.142559
2023-02-01 17:00:00    0.142425
2023-02-01 18:00:00    0.142625
2023-02-01 19:00:00    0.142386
2023-02-01 20:00:00    0.142488
2023-02-01 21:00:00    0.142263
2023-02-01 22:00:00    0.142144
2023-02-01 23:00:00    0.142217
2023-02-02 00:00:00    0.142204
2023-02-02 01:00:00    0.141977
2023-02-02 02:00:00    0.142426
2023-02-02 03:00:00    0.142249
2023-02-02 04:00:00    0.142233
2023-02-02 05:00:00    0.142290
2023-02-02 06:00:00    0.142448
Freq: h, Name: predicted_mean, dtype: fl

In [23]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

# Calculate RMSE and MAPE
rmse = np.sqrt(mean_squared_error(actuals, sarima_forecast_values))
mape = mean_absolute_percentage_error(actuals, sarima_forecast_values) * 100

# Display
print(f"RMSE: {rmse:.6f}")
print(f"MAPE: {mape:.2f}%")


RMSE: 0.005164
MAPE: 3.11%


In [26]:
# Import the function
# from src.metric_logging import log_experiment

# Define parameters and metrics
params = {
    "order": "(1, 1, 1)",
    "seasonal_order": "(1, 1, 1, 24)",  # SARIMA seasonal params
    "seasonality": "24 hours (daily seasonality)"
}

metrics = {
    "RMSE": rmse,
    "MAPE": mape
}

# Log experiment
log_experiment("SARIMA_model_for_hourly_data", model_object=sarima_result, params=params, metrics=metrics)


2025/04/18 00:17:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'SARIMA_model_for_hourly_data_Model'.
2025/04/18 00:35:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: SARIMA_model_for_hourly_data_Model, version 1
Created version '1' of model 'SARIMA_model_for_hourly_data_Model'.


✅ SARIMA_model_for_hourly_data registered and logged successfully to DagsHub MLflow.
🏃 View run SARIMA_model_for_hourly_data-Baseline at: https://dagshub.com/vamsisaigarapati/bitcoin_price_pred_CSE574.mlflow/#/experiments/0/runs/781248fa6e1c46c3b19ecd690dc150f9
🧪 View experiment at: https://dagshub.com/vamsisaigarapati/bitcoin_price_pred_CSE574.mlflow/#/experiments/0
